In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
import preprocess_utils.session2vec as sess2vec
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

Using TensorFlow backend.


In [3]:
train_df = data.train_df('small')

In [13]:
train_df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
15932992,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,2.0
15932994,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0
15932995,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0
15932996,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0
15932997,004A07DM0IDW,1d688ec168932,1541555792,6,clickout item,3241426,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0
15932998,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0
15932999,009RGHI3G9A3,f05ab0de907e2,1541570935,1,interaction item info,7065316,IN,"Nathdwara, India",mobile,NaN,NaN,NaN,1.0
15933000,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28,1.0
15933001,00Y1Z24X8084,26b6d294d66e7,1541651766,1,clickout item,3843244,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,1.0
15933002,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,1.0


In [4]:
res_df = train_df[['user_id','session_id','prices']]
res_df = res_df[train_df.action_type == 'clickout item']

In [5]:
res_df.head()

,user_id,session_id,prices
13,00RL8Z82B2Z1,aff3928535f48,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
15,00RL8Z82B2Z1,aff3928535f48,162|25|150|143|101|49|118|131|18|100|101|143|5...
115,02SRUT1NQYH1,3599a6f709eab,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
122,03K8AXBL4BX2,ec139e10b9238,104|92|100|103|102|104|72|85|81|75|107|86|98|8...


In [6]:
expanded_prices = res_df.prices.str.split('|', expand=True).fillna(0).astype('int')

In [7]:
expanded_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
13,95,66,501,112,95,100,101,72,82,56,56,143,70,25,71,162,73,143,188,118,77,131,143,49,165
15,162,25,150,143,101,49,118,131,18,100,101,143,51,143,123,36,66,188,124,138,94,137,180,112,118
115,64,54,36,121,76,81,92,40,73,52,98,104,56,414,67,111,21,122,55,104,56,64,40,29,44
121,104,92,100,103,102,104,72,85,81,75,107,86,98,83,98,103,105,99,106,99,98,96,96,105,99
122,104,92,100,103,102,104,72,85,81,75,107,86,98,83,98,103,105,99,106,99,98,96,96,105,99
176,178,104,110,94,57,96,46,61,48,35,50,38,59,44,57,56,55,45,146,41,44,44,39,38,126
177,178,104,110,94,57,96,46,61,48,35,50,38,59,44,57,56,55,45,146,41,44,44,39,38,126
180,51,43,69,49,62,50,55,42,87,46,43,114,194,50,196,351,22,154,116,89,55,270,192,92,26
181,54,39,55,51,41,94,24,38,59,24,72,314,46,40,183,51,50,129,96,118,86,125,51,50,117
184,36,45,99,48,49,50,49,56,40,63,35,95,40,78,132,48,76,62,58,43,53,126,65,42,45


In [8]:
#scale log
log_prices = np.log(expanded_prices.values + 1)

In [9]:
#scale min-max
max_price = np.max(log_prices)
min_price = np.min(log_prices)

In [14]:
log_prices = (log_prices - min_price) / (max_price - min_price)

In [15]:
pd.DataFrame(log_prices)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
1,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
2,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
3,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
4,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
5,0.453223,0.365596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.294020,0.394942,0.000000
7,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.294020,0.394942,0.000000
8,0.307609,0.319685,0.330551,0.335602,0.340428,0.340428,0.365596,0.365596,0.365596,0.372836,0.372836,0.382866,0.382866,0.389071,0.389071,0.392046,0.394942,0.394942,0.408363,0.408363,0.413299,0.413299,0.418020,0.418020,0.420306
9,0.307609,0.319685,0.330551,0.335602,0.340428,0.340428,0.365596,0.365596,0.365596,0.372836,0.372836,0.382866,0.382866,0.389071,0.389071,0.392046,0.394942,0.394942,0.408363,0.408363,0.413299,0.413299,0.418020,0.418020,0.420306


In [16]:
for i in range(25):
    res_df['price_{}'.format(i)] = log_prices[:,i]

In [17]:
res_df

,user_id,session_id,prices,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,price_20,price_21,price_22,price_23,price_24
15932994,004A07DM0IDW,1d688ec168932,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932995,004A07DM0IDW,1d688ec168932,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932996,004A07DM0IDW,1d688ec168932,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932997,004A07DM0IDW,1d688ec168932,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932998,004A07DM0IDW,1d688ec168932,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15933000,009RGHI3G9A3,f05ab0de907e2,64|28,0.453223,0.365596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15933001,00Y1Z24X8084,26b6d294d66e7,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.294020,0.394942,0.000000
15933002,00Y1Z24X8084,26b6d294d66e7,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.294020,0.394942,0.000000
15933006,01V3WDTDM5CU,07628a0f5be0b,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...,0.307609,0.319685,0.330551,0.335602,0.340428,0.340428,0.365596,0.365596,0.365596,0.372836,0.372836,0.382866,0.382866,0.389071,0.389071,0.392046,0.394942,0.394942,0.408363,0.408363,0.413299,0.413299,0.418020,0.418020,0.420306
15933007,01V3WDTDM5CU,07628a0f5be0b,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...,0.307609,0.319685,0.330551,0.335602,0.340428,0.340428,0.365596,0.365596,0.365596,0.372836,0.372836,0.382866,0.382866,0.389071,0.389071,0.392046,0.394942,0.394942,0.408363,0.408363,0.413299,0.413299,0.418020,0.418020,0.420306


In [19]:
train_df.merge(res_df.drop(['user_id','session_id','prices'], axis=1), left_index=True, right_index=True)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,price_20,price_21,price_22,price_23,price_24
15932994,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932995,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932996,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932997,004A07DM0IDW,1d688ec168932,1541555792,6,clickout item,3241426,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932998,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15933000,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28,1.0,0.453223,0.365596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15933001,00Y1Z24X8084,26b6d294d66e7,1541651766,1,clickout item,3843244,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,1.0,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.294020,0.394942,0.000000
15933002,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,1.0,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.294020,0.394942,0.000000
15933006,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,clickout item,2817590,PL,"Wisla, Poland",mobile,Sort by Price,3565720|2947584|4115018|2039671|3836538|801409...,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...,1.0,0.307609,0.319685,0.330551,0.33

In [4]:
from extract_features.rnn.clickout_vector_prices import ClickoutVectorPrices
feat = ClickoutVectorPrices()

In [5]:
f = feat.read_feature()
feat_cols = f.columns

clickout_vector_prices feature read


In [6]:
f

,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,price_20,price_21,price_22,price_23,price_24
13,0.495562,0.456514,0.675169,0.513264,0.495562,0.501075,0.502145,0.465826,0.479764,0.438964,0.438964,0.539585,0.462810,0.353739,0.464328,0.553041,0.467303,0.539585,0.569109,0.518881,0.473019,0.530138,0.539585,0.424738,0.555021
15,0.553041,0.353739,0.544738,0.539585,0.502145,0.424738,0.518881,0.530138,0.319685,0.501075,0.502145,0.539585,0.428996,0.539585,0.523350,0.392046,0.456514,0.569109,0.524222,0.535748,0.494426,0.534964,0.564414,0.513264,0.518881
115,0.453223,0.435086,0.392046,0.521584,0.471618,0.478448,0.492115,0.403192,0.467303,0.431064,0.498903,0.505292,0.438964,0.654505,0.458122,0.512299,0.335602,0.522471,0.437042,0.505292,0.438964,0.453223,0.403192,0.369276,0.413299
121,0.505292,0.492115,0.501075,0.504253,0.503204,0.505292,0.465826,0.483619,0.478448,0.470198,0.508350,0.484875,0.498903,0.481065,0.498903,0.504253,0.506321,0.499995,0.507340,0.499995,0.498903,0.496688,0.496688,0.506321,0.499995
122,0.505292,0.492115,0.501075,0.504253,0.503204,0.505292,0.465826,0.483619,0.478448,0.470198,0.508350,0.484875,0.498903,0.481065,0.498903,0.504253,0.506321,0.499995,0.507340,0.499995,0.498903,0.496688,0.496688,0.506321,0.499995
176,0.563207,0.505292,0.511325,0.494426,0.440852,0.496688,0.418020,0.448093,0.422544,0.389071,0.426888,0.397762,0.444533,0.413299,0.440852,0.438964,0.437042,0.415685,0.541823,0.405808,0.413299,0.413299,0.400511,0.397762,0.525945
177,0.563207,0.505292,0.511325,0.494426,0.440852,0.496688,0.418020,0.448093,0.422544,0.389071,0.426888,0.397762,0.444533,0.413299,0.440852,0.438964,0.437042,0.415685,0.541823,0.405808,0.413299,0.413299,0.400511,0.397762,0.525945
180,0.428996,0.410859,0.461270,0.424738,0.449830,0.426888,0.437042,0.408363,0.486115,0.418020,0.410859,0.515169,0.572502,0.426888,0.573610,0.636629,0.340428,0.547577,0.517041,0.488555,0.437042,0.608236,0.571383,0.492115,0.357837
181,0.435086,0.400511,0.437042,0.428996,0.405808,0.494426,0.349481,0.397762,0.444533,0.349481,0.465826,0.624571,0.418020,0.403192,0.566198,0.428996,0.426888,0.528480,0.496688,0.518881,0.484875,0.525087,0.428996,0.426888,0.517965
184,0.392046,0.415685,0.499995,0.422544,0.424738,0.426888,0.424738,0.438964,0.403192,0.451540,0.389071,0.495562,0.403192,0.474402,0.530957,0.422544,0.471618,0.449830,0.442708,0.410859,0.433094,0.525945,0.454881,0.408363,0.415685


In [7]:
joined = feat.join_to(train_df)
joined

clickout_vector_prices feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,price_9,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,price_20,price_21,price_22,price_23,price_24
15932992,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15932994,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932995,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932996,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932997,004A07DM0IDW,1d688ec168932,1541555792,6,clickout item,3241426,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932998,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,1.0,0.462810,0.418020,0.422544,0.471618,0.454881,0.454881,0.507340,0.456514,0.486115,0.410859,0.431064,0.413299,0.446328,0.448093,0.426888,0.437042,0.379624,0.459707,0.413299,0.353739,0.403192,0.369276,0.580030,0.449830,0.451540
15932999,009RGHI3G9A3,f05ab0de907e2,1541570935,1,interaction item info,7065316,IN,"Nathdwara, India",mobile,NaN,NaN,NaN,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15933000,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28,1.0,0.453223,0.365596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15933001,00Y1Z24X8084,26b6d294d66e7,1541651766,1,clickout item,3843244,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,1.0,0.468760,0.294020,0.340428,0.397762,0.437042,0.413299,0.365596,0.386013,0.345049,0.361786,0.278483,0.509351,0.325254,0.335602,0.392046,0.325254,0.325254,0.392046,0.340428,0.382866,0.386013,0.372836,0.2940